# Radiative-Convective Equilibrium with CAM3 scheme

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import climlab
from climlab import constants as const

## Create a new custom process for RCE

In [126]:
class RCEmodel(climlab.TimeDependentProcess):
    def __init__(self,
                 num_lev=30,
                 num_lat=1,
                 lev=None,
                 lat=None,
                 water_depth=1.0,
                 albedo_sfc=0.299,
                 timestep=1. * const.seconds_per_day,
                 # lapse rate for convective adjustment, in K / km
                 adj_lapse_rate=6.5,
                 # insolation
                 Q=341.3,
                 CO2=380.,
                 **kwargs):
        # Check to see if an initial state is already provided
        #  If not, make one
        if 'state' not in kwargs:
            state = climlab.column_state(num_lev, num_lat, lev, lat, water_depth)
            kwargs.update({'state': state})
        super(RCEmodel, self).__init__(timestep=timestep, **kwargs)
        self.param['water_depth'] = water_depth
        self.param['albedo_sfc'] = albedo_sfc
        self.param['Q'] = Q
        
        self.param['adj_lapse_rate'] = adj_lapse_rate
        #self.add_subprocess('convadj', 
        #    climlab.convection.convadj.ConvectiveAdjustment(state=self.state, **self.param))

        #  Initialize specific humidity
        #h2o = climlab.radiation.water_vapor.ManabeWaterVapor(state=self.state, **self.param)
        #self.add_subprocess('H2O', h2o)
        #self.q = self.subprocess['H2O'].q
        rad = climlab.radiation.CAM3Radiation(state=self.state, 
                                              #q=self.q, 
                                              CO2=CO2,
                                              insolation=Q,
                                              aldif=albedo_sfc,
                                              aldir=albedo_sfc,
                                              asdif=albedo_sfc,
                                              asdir=albedo_sfc)
        self.add_subprocess('rad', rad)

In [127]:
rce = RCEmodel()


 Building _cam3_radiation.so ... 

f2py -c -m _cam3_radiation --fcompiler=gnu95 --noopt -I/Users/Brian/climlab/src/radiation/cam3/src --f77flags="-DPLEV=30 -ffixed-line-length-132 -fdefault-real-8" --f90flags="-DPLEV=30 -fdefault-real-8 -fno-range-check -ffree-form" _cam3_radiation.pyf /Users/Brian/climlab/src/radiation/cam3/src/CliMT_dimensions.F90 /Users/Brian/climlab/src/radiation/cam3/src/ppgrid.F90 /Users/Brian/climlab/src/radiation/cam3/src/pmgrid.F90 /Users/Brian/climlab/src/radiation/cam3/src/prescribed_aerosols.F90 /Users/Brian/climlab/src/radiation/cam3/src/quicksort.F90 /Users/Brian/climlab/src/radiation/cam3/src/abortutils.F90 /Users/Brian/climlab/src/radiation/cam3/src/shr_kind_mod.F90 /Users/Brian/climlab/src/radiation/cam3/src/absems.F90 /Users/Brian/climlab/src/radiation/cam3/src/wv_saturation.F90 /Users/Brian/climlab/src/radiation/cam3/src/aer_optics.F90 /Users/Brian/climlab/src/radiation/cam3/src/cmparray_mod.F90 /Users/Brian/climlab/src/radiation/cam3/src/shr_const_

In [128]:
rce.state

{'Tatm': Field([ 200.        ,  202.68965517,  205.37931034,  208.06896552,
         210.75862069,  213.44827586,  216.13793103,  218.82758621,
         221.51724138,  224.20689655,  226.89655172,  229.5862069 ,
         232.27586207,  234.96551724,  237.65517241,  240.34482759,
         243.03448276,  245.72413793,  248.4137931 ,  251.10344828,
         253.79310345,  256.48275862,  259.17241379,  261.86206897,
         264.55172414,  267.24137931,  269.93103448,  272.62068966,
         275.31034483,  278.        ]), 'Ts': Field([ 288.])}

In [129]:
rce.compute()

In [130]:
rce.step_forward()

In [131]:
rce.state

{'Tatm': Field([ 199.9993146 ,  202.6893184 ,  205.37909842,  208.0688172 ,
         210.75851198,  213.44819277,  216.13786025,  218.82752889,
         221.51719323,  224.2068549 ,  226.89650833,  229.58616292,
         232.27581919,  234.96547566,  237.65513006,  240.34478341,
         243.03443567,  245.72408719,  248.41373194,  251.1033778 ,
         253.79302659,  256.48267471,  259.17232177,  261.86196807,
         264.55161459,  267.24126312,  269.93091668,  272.62058994,
         275.31020436,  277.999891  ]), 'Ts': Field([ 286.00529166])}

In [132]:
rce.integrate_years()

Integrating for 365 steps, 365.2422 days, or 1.0 years.
Total elapsed time is 1.00207478763 years.


In [133]:
rce.Ts

Field([ 266.1361462])

In [134]:
rce.Tatm

Field([ 199.75039872,  202.56630551,  205.30095789,  208.01330573,
        210.71690958,  213.41539205,  216.10899509,  218.80296399,
        221.49532867,  224.18667839,  226.87498654,  229.56364349,
        232.25284022,  234.94203148,  237.63036244,  240.31817768,
        243.00545947,  245.69227583,  248.37639381,  251.06059382,
        253.74544391,  256.42946104,  259.11271002,  261.79457746,
        264.47456121,  267.15109974,  269.82142117,  272.47639893,
        275.13470473,  277.73429311])

In [108]:
rce.subprocess.rad.aldif

Field([ 0.299])

In [114]:
rce.subprocess.rad.flus

Field([-99.])

In [116]:
rce.subprocess.rad.Output.keys()

['lwuflx',
 'SrfRadFlx',
 'swflx',
 'LwSrfCf',
 'SwSrf',
 'SwToaCf',
 'LwSrf',
 'SwSrfCf',
 'lwflx',
 'swhr',
 'lwdflx',
 'TdotRad',
 'LwToaCf',
 'SwToa',
 'LwToa',
 'lwhr']

In [135]:
rce.subprocess.rad.Output['LwToa']

array([[-257.06125055]])

In [118]:
const.sigma * rce.Ts**4

Field([ 288.47556621])

In [136]:
rce.subprocess.rad.Output['SwToa']

array([[ 234.67466759]])

In [123]:
rce.integrate_years()

Integrating for 365 steps, 365.2422 days, or 1.0 years.
Total elapsed time is 4.00008542277 years.


In [139]:
rce

AttributeError: 'RCEmodel' object has no attribute 'q'

In [140]:
rce.subprocess.rad.q

Field([  1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08,
         1.00000000e-08,   1.00000000e-08,   1.00000000e-08])

In [141]:
rce.subprocess.rad.CO2 = 600.

In [142]:
rce.state

{'Tatm': Field([ 199.75039872,  202.56630551,  205.30095789,  208.01330573,
         210.71690958,  213.41539205,  216.10899509,  218.80296399,
         221.49532867,  224.18667839,  226.87498654,  229.56364349,
         232.25284022,  234.94203148,  237.63036244,  240.31817768,
         243.00545947,  245.69227583,  248.37639381,  251.06059382,
         253.74544391,  256.42946104,  259.11271002,  261.79457746,
         264.47456121,  267.15109974,  269.82142117,  272.47639893,
         275.13470473,  277.73429311]), 'Ts': Field([ 266.1361462])}

In [143]:
rce.integrate_years()

Integrating for 365 steps, 365.2422 days, or 1.0 years.
Total elapsed time is 2.00141166601 years.


In [144]:
rce.state

{'Tatm': Field([ 199.46156362,  202.43503542,  205.22079681,  207.95762638,
         210.6756725 ,  213.38302738,  216.08049781,  218.77859195,
         221.4734666 ,  224.16631321,  226.85312716,  229.54061656,
         232.22916981,  234.9177001 ,  237.60451235,  240.29026641,
         242.97496311,  245.65868565,  248.33704884,  251.01551637,
         253.69522911,  256.37325697,  259.04970979,  261.72336797,
         264.39330748,  267.05645376,  269.7074574 ,  272.32909605,
         274.95691132,  277.46940592]), 'Ts': Field([ 266.91411486])}

In [145]:
rce.subprocess.rad.Output['SwToa'] + rce.subprocess.rad.Output['LwToa']

array([[-23.10401892]])

In [ ]:
## BUGGY!!!